In [1]:
import numpy as np
import sys
import copy
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from threading import Thread
from time import sleep
import datetime

In [2]:
np.set_printoptions( threshold = sys.maxsize )

In [3]:
class Object2D:
    def __init__( self, sizeX_, sizeY_, pattern_ ):
        self.sizeX = sizeX_
        self.sizeY = sizeY_
        self.pattern = pattern_

In [4]:
class Mesh( Object2D ):
    def __init__( self, sizeX_, sizeY_, pattern_ ):
        super().__init__( sizeX_, sizeY_, pattern_ )
    def Copy( self ):
        return Mesh( self.sizeX, self.sizeY, copy.deepcopy( self.pattern ) )
    def Scale( self, xScale, yScale ):
        return self
    def AddMass( self, mass ):
        for x in range( len( self.pattern ) ):
            for y in range( len( x ) ):
                self.pattern[ x ][ y ] *= mass
        return self
    def ToMass( self, x, y ):
        return Mass( x, y, self )

In [5]:
class Planet( Mesh ):
    def __init__( self, radius_, distributedMass_, default_ = 0.0, ringDistance_ = 0.0, ringDepth_ = 0.0, increment = .01 ):
        self.radius = radius_
        self.ringDistance = ringDistance_
        self.default = default_
        self.ringDepth = ringDepth_
        self.ModifyMass( distributedMass_ )
        self.FillPlanet( self.radius, True, increment )
    def ModifyMass( self, distributedMass_ ):
        self.distributedMass = distributedMass_
        self.mass = ( 2.0 * np.pi * ( self.radius * self.radius ) ) * self.distributedMass
        return self
    def ChangeRadius( self, radius_, fillRing, increment = .01, ):
        self.FillPlanet( radius_, fillRing, increment )
        return self
    def FillPlanet( self, radius_, doFillRing = True, increment = .01, distributedMass_ = None ):
        trueRadius = int( self.radius + self.ringDistance + self.ringDepth ) * 2
        self.radius = radius_
        self.sizeX = trueRadius
        self.sizeY = trueRadius
        planet = []
        if distributedMass_ != None:
            self.ModifyMass( distributedMass_ )
        for i in range( self.sizeX ):
            planet.append( [ self.default ] * self.sizeY )
        for i in range( int( radius_ ) ):
            currentRadius = radius_ - i
            theta = 0.0
            while theta <= ( 2.0 * np.pi ):
                planet[ 
                    ( int( currentRadius ) - int( np.sin( theta ) * currentRadius ) ) + 
                    int( ( self.sizeX / 2.0 ) - ( currentRadius ) ) ][ int( currentRadius ) - 
                    int( np.cos( theta ) * currentRadius ) + 
                    int( ( self.sizeY / 2.0 ) - ( currentRadius ) ) ] = self.distributedMass
                theta += increment
        self.pattern = planet
        if doFillRing == True:
            self.FillRing( self.ringDistance, self.ringDepth, increment )
        return self
    
    def FillRing( self, ringDistance_ = None, ringDepth_ = None, increment = .01, fillPlanet = False ):
        if ringDistance_ == None:
            ringDistance_ = self.ringDistance
        else:
            self.ringDistance = ringDistance_
        if ringDepth_ == None:
            ringDepth_ = self.ringDepth
        else:
            self.ringDepth = ringDepth_
        trueRadius = int( self.radius + ringDistance_ + ringDepth_ ) * 2
        for i in range( int( ringDepth_ ) ):
            theta = 0.0
            while theta <= ( 2.0 * np.pi ):
                self.pattern[ 
                    ( int( i ) - int( np.sin( theta ) * ( ringDistance_ + self.radius + i ) ) ) + 
                    int( ( self.sizeX / 2.0 ) - ( i ) ) ][ int( i ) - 
                    int( np.cos( theta ) * ( ringDistance_ + self.radius + i ) ) + 
                    int( ( self.sizeY / 2.0 ) - ( i ) ) ] = self.distributedMass
                theta += increment
        return self

In [6]:
class Mass( Object2D ):
    def __init__( self, x_, y_, mesh, noDeepCopy = False ):
        if noDeepCopy == False:
            super().__init__( mesh.sizeX, mesh.sizeY, copy.deepcopy( mesh.pattern ) )
        else:
            super().__init__( mesh.sizeX, mesh.sizeY, mesh.pattern )
        self.x = x_
        self.y = y_
    def ToMesh( self ):
        return Mesh( self.sizeX, self.sizeY, copy.deepcopy( self.pattern ) )
    def Copy( self ):
        return Mass( self.x, self.y, self.ToMesh(), True )

In [7]:
class GravitySimulationFeild:
    def __init__( self, size, boundry_, default_ = 0, phi_ = [], masses_ = [] ):
        self.phi = phi_
        self.masses = masses_
        self.boundry = boundry_
        self.default = default_
        if len( self.phi ) == 0:
            self.CreatePhi( size ).CreateBoundry( self.boundry ).SetDefault( self.default )
        self.size = size
    def MakeObject( self, mesh, x, y ):
        self.masses.append( mesh.ToMass( x, y ) )
        return self
    def AddMass( self, mass ):
        self.masses.append( mass )
    def CreateZeros( self, size ):
        return np.zeros( [ size + 1, size + 1 ] )        
    def CreatePhi( self, size ):
        self.phi = self.CreateZeros( size )
        #Fluid A.P.I
        return self
    def CreateBoundry( self, boundry ):
        size = len( self.phi ) - 1
        for i in range( size ):
            self.phi[ i ][ 0 ] = boundry
            self.phi[ 0 ][ i ] = boundry
            self.phi[ i ][ size ] = boundry
            self.phi[ size ][ i ] = boundry
        return self
    def SetDefault( self, default ):
        for i in range( 1, len( self.phi ) - 1 ):
            for j in range( 1, len( self.phi[ i ] ) - 1 ):
                self.phi[ i ][ j ] = default
        return self
    def AddMassesToFeild( self ):
        massfulPhi = copy.deepcopy( self.phi )
        for i in self.masses:
            for x in range( i.sizeX ):
                for y in range( i.sizeY ):
                    massfulPhi[ i.x + x ][ i.y + y ] = i.pattern[ x ][ y ]
        return GravitySimulationFeild( self.size, self.boundry, self.default, massfulPhi, copy.deepcopy( self.masses ) )
    def Calculate( self, maximumExecutions, noDeepCopy = False, xStart = 1, yStart = 1, xEnd_ = None, yEnd_ = None ):
        phiBase = self.phi
        if xEnd_ == None:
            xEnd_ = self.size
            yEnd_ = self.size
        if noDeepCopy == False:
            phiBase = copy.deepcopy( self.phi )
        phiCalculate = self.CreateZeros( self.size )
        for execution in range( maximumExecutions ):
            for x in range( xStart, xEnd_ ):
                for y in range( yStart, yEnd_ ):
                    phiCalculate[ x ][ y ] = .25 * ( phiBase[ x + 1 ][ y ] + phiBase[ x - 1 ][ y ] + phiBase[ x ][ y + 1 ] + phiBase[ x ][ y - 1 ] )
            for i in range( xStart, xEnd_ ):
                for j in range( yStart, yEnd_ ):
                    phiBase[ i ][ j ] = phiCalculate[ i ][ j ]
        return GravitySimulationFeild( self.size, self.boundry, self.default, phiBase, copy.deepcopy( self.masses ) )
    def MultiThreadedCalculate( self, amountOfThreads, executionDepth ):
        threadCopy = GravitySimulationFeild( self.size, self.boundry, self.default, copy.deepcopy( self.phi ), copy.deepcopy( self.masses ) )
        segmentSize = threadCopy.size / amountOfThreads
        threads = []
        for i in range( amountOfThreads ):
            threads.append( Thread( target = threadCopy.Calculate, args = ( executionDepth, True, int( i * segmentSize ), 1, int( ( i + 1 ) * segmentSize ), threadCopy.size ) ) )
            threads[ -1 ].start()
        for i in threads:
            i.join()
        return threadCopy

In [8]:
g = GravitySimulationFeild( 80, -4.0, 3.0 )

In [ ]:
startTime = datetime.datetime.now()
plt.imshow( g.Calculate( 10000 ).phi )
print( datetime.datetime.now() - startTime )

In [ ]:
startTime = datetime.datetime.now()
plt.imshow( g.MultiThreadedCalculate( 4, 10000 ).phi )
print( datetime.datetime.now() - startTime )

In [ ]:
g0 = GravitySimulationFeild( 80, -4.0, 3.0 )

In [ ]:
startTime = datetime.datetime.now()
plt.imshow( g.MultiThreadedCalculate( 2, 10000 ).phi )
print( datetime.datetime.now() - startTime )